In [19]:
import sys
home=os.path.expanduser("~")
sys.path.append(os.path.join(os.path.expanduser("~"), 'station2grid'))

import os
import pandas as pd
from sklearn.metrics import mean_squared_error
from tools import CustomKNN, plotMap, CommonObj
from tools.options import Opt
from models import grid2code_model, station2code_model, station2gridSD_model
import matplotlib.pyplot as plt

# option

In [3]:
opt=Opt()
epa_station_info = CommonObj().epa_station_info

In [26]:
# epa_station_info.head(2)

In [5]:
opt1 = Opt(
    domain='air', 
    k=5,
    weightKNN='distance',
    ae_type='A1',
    code_length=4576,
    batch_size=2,  
    n_epochs=10, 
    model_name='grid2code',
#     features='pm25', 
#     val_stations='Shilin_Guting',
#     epa_station_path=os.path.join(home, 'station2grid', 'datasets', 'npy', 'epa', 'epa2014.npy') 
)

opt2 = Opt(
    domain='air', 
    k=5,
    weightKNN='distance',
    ae_type='A1',
    code_length=4576,    
    batch_size=2,  
    n_epochs=10, 
    model_name='station2code',
    features='pm25', 
#     val_stations='Shilin_Guting', 
#     epa_station_path=os.path.join(home, 'station2grid', 'datasets', 'npy', 'epa', 'epa2014.npy') 
)

opt3 = Opt(
    domain='air', 
    k=5,
    weightKNN='distance',
    ae_type='A1',
    code_length=4576,
    batch_size=2,  
    n_epochs=10, 
    model_name='station2gridSD',
    features='pm25', 
#     val_stations='Shilin_Guting', 
    epa_station_path=os.path.join(home, 'station2grid', 'datasets', 'npy', 'epa', 'epa2014.npy') 
)

# g2c

In [6]:

# grid2code = grid2code_model.Grid2Code(opt1)
# grid2code.train()
# grid2code.test()


# s2c

In [7]:

for x in epa_station_info.SiteEngName[:2]:
    print('-'*80)
    print(x)
    
    opt2.val_stations = x
    station2code = station2code_model.Station2Code(opt2)
    station2code.train()

--------------------------------------------------------------------------------
Yangming
training...


/media/disk3/feynman52/.pyenv/versions/3.6.5/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


finish!
--------------------------------------------------------------------------------
Wanli
training...
finish!


# s2gSD

In [8]:
result = pd.DataFrame()

for x in epa_station_info.SiteEngName[:2]:
    print('-'*80)
    print(x)
    
    opt3.val_stations = x
    
    station2GridSD = station2gridSD_model.Station2GridSD(opt3)
    grids = station2GridSD.test() ###
    stations = station2GridSD.stations
    stations = station2GridSD.dataGenerator.denormalize(stations)
    
    df = epa_station_info[epa_station_info.SiteEngName==x]
    indexs = df.index.values
    rows = df.row.values
    cols = df.col.values
    
    dummy = pd.DataFrame()
    dummy['real'] = stations[:,indexs,0].reshape(-1)
    dummy['predict'] = grids[:,rows,cols,0].reshape(-1)
    dummy['station'] = x
    result = result.append(dummy)

--------------------------------------------------------------------------------
Yangming
testing...
finish!
--------------------------------------------------------------------------------
Wanli
testing...
finish!


# plot

In [25]:
# i = 30
# lats = epa_station_info[epa_station_info.SiteEngName!=x].lat.values
# lons = epa_station_info[epa_station_info.SiteEngName!=x].lon.values
# vals = stations[i,...].reshape(-1)
# title = 'station'
# fig_num = 1
# plotMap(lats,lons,vals,title,fig_num)
# plt.show()



# customKNN = CustomKNN()
# customKNN.coordinates.shape
# df = epa_station_info[epa_station_info.SiteEngName==x]
# indexs = df.index.values
# rows = df.row.values
# cols = df.col.values

# lats = customKNN.coordinates[:,0]
# lons = customKNN.coordinates[:,1]
# vals = grids[i,...].reshape(-1)
# title = 'grid'
# fig_num = 1
# plotMap(lats,lons,vals,title,fig_num)
# plt.show()


# mse

In [11]:
result

real    predict   station
0   31.0   9.730068  Yangming
1   26.0   9.027493  Yangming
2   22.0   8.495272  Yangming
3   28.0   9.348173  Yangming
4   27.0   9.420323  Yangming
5   25.0   9.649852  Yangming
6   27.0  11.074162  Yangming
7   26.0  11.339284  Yangming
8   29.0  11.671167  Yangming
9   29.0  11.538035  Yangming
10  26.0  12.890333  Yangming
11  24.0  13.103138  Yangming
12  24.0  13.700422  Yangming
13  25.0  14.757623  Yangming
14  25.0  14.744782  Yangming
15  22.0  13.269906  Yangming
16  22.0  13.484660  Yangming
17  23.0  13.962692  Yangming
18  22.0  12.822923  Yangming
19  25.0  10.348546  Yangming
20  25.0   7.722846  Yangming
21  22.0   7.255434  Yangming
22  21.0   7.396238  Yangming
23  23.0   8.231689  Yangming
24  24.0   8.551249  Yangming
25  19.0   8.694180  Yangming
26  19.0   9.025151  Yangming
27  21.0   9.682240  Yangming
28  21.0  11.029013  Yangming
29  23.0  11.926946  Yangming
..   ...        ...       ...
18  26.0  14.389931     Wanli
19  24.0  12.549226     Wanli
20  20.0   9.466084     Wanli
21  23.0   9.125181     Wanli
22  22.0   9.182009     Wanli
23  23.0  10.029366     Wanli
24  25.0   9.303833     Wanli
25  25.0   9.438560     Wanli
26  24.0   9.358495     Wanli
27  22.0  10.464197     Wanli
28  24.0  11.302309     Wanli
29  26.0  12.253785     Wanli
30  26.0  12.437929     Wanli
31  29.0  12.614181     Wanli
32  34.0  12.420284     Wanli
33  36.0  11.948646     Wanli
34  38.0  11.936064     Wanli
35  40.0  11.747632     Wanli
36  42.0  11.324885     Wanli
37  42.0  12.888263     Wanli
38  40.0  12.645581     Wanli
39  37.0  12.193266     Wanli
40  33.0  12.259893     Wanli
41  36.0  10.677469     Wanli
42  34.0   9.140641     Wanli
43  40.0   6.711542     Wanli
44  47.0   5.915312     Wanli
45  44.0   6.051478     Wanli
46  45.0   6.144004     Wanli
47  46.0   6.401850     Wanli

[96 rows x 3 columns]

In [24]:
mean_squared_error(result.real,result.predict)

372.949616079039